In [1]:
# 1.获取数据集
# 2.基本数据处理
# 2.1 缩小数据范围
# 2.2 选择时间特征
# 2.3 去掉签到较少的地方
# 2.4 确定特征值和目标值
# 2.5 分割数据集
# 3.特征工程 -- 特征预处理(标准化)
# 4.机器学习 -- knn+cv
# 5.模型评估

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# 1. 获取数据集
data =  pd.read_csv('../data/FBlocation/train.csv')

In [4]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [5]:
data.shape

(29118021, 6)

In [6]:
# 2.基本数据处理
# 2.1 缩小数据范围
facebook_data = data.query('x > 2.0 & x < 3.0 & y > 2.0 & y < 3.0')
facebook_data.shape

(314785, 6)

In [7]:
# 2.2 选择时间特征
time = pd.to_datetime(facebook_data["time"], unit='s')
time = pd.DatetimeIndex(time)
facebook_data["day"] = time.day
facebook_data["hour"] = time.hour
facebook_data["weekday"] = time.weekday
facebook_data.head()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

,row_id,x,y,accuracy,time,place_id,day,hour,weekday
82,82,2.9265,2.3590,55,415119,9313893105,5,19,0
159,159,2.4166,2.7762,66,74327,3432339087,1,20,3
163,163,2.1663,2.3755,84,669737,3869813743,8,18,3
191,191,2.7878,2.5728,144,330072,7194654578,4,19,6
310,310,2.3695,2.2034,3,234719,2636621520,3,17,5


In [8]:
# 2.3 去掉签到较少的地方
place_count = facebook_data.groupby("place_id").count()
place_count = place_count[place_count["row_id"]>3]
facebook_data = facebook_data[facebook_data["place_id"].isin(place_count.index)]

In [9]:
# 2.4 确定特征值和目标值
x = facebook_data[["x", "y", "accuracy", "day", "hour", "weekday"]]
y = facebook_data["place_id"]

In [10]:
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)

In [13]:
# 3.特征工程 -- 特征预处理(标准化)
transform = StandardScaler()
x_train = transform.fit_transform(x_train)
x_test = transform.transform(x_test)

In [15]:
# 4.机器学习 -- knn+cv
# 创建评估器模型
estimator = KNeighborsClassifier()

# 网格搜索，交叉验证
param = {"n_neighbors":[1,3,5,7,9]}
estimator = GridSearchCV(estimator,param_grid=param, cv=5)

estimator.fit(x_train, y_train)

c:\program files\python36\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [ ]:
# 5.模型评估
score = estimator.score(x_test, y_test)
print("最后预测的准确率为:\n", score)

print('交叉验证的最好的结果', estimator.best_score_)
print('交叉验证的最好的参数模型', estimator.best_estimator)